In [1]:
import os
import numpy as np
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose
from hydra.utils import instantiate
from starfish.data import StarfishDataModule
from starfish.model import FasterRCNNLightning

import torch
from pprint import pprint
# Set the environment variable HYDRA_FULL_ERROR=1 to see the full stack trace
os.environ['HYDRA_FULL_ERROR'] = '1'

os.environ['PROJECT_ROOT'] = '/Users/moustholmes/starfishDetection'

# Change the current working directory to the project root
os.chdir('/Users/moustholmes/starfishDetection')

initialize(version_base=None, config_path='configs', job_name='notebook')

cfg_train = compose(config_name='main_config',overrides=['data.data_from_bucket=False'])

print(OmegaConf.to_yaml(cfg_train))



model:
  _target_: starfish.model.FasterRCNNLightning
  num_classes: 2
  log_ap: true
  optimizer:
    _target_: torch.optim.SGD
    _partial_: true
    lr: 0.0001
  scheduler:
    _target_: torch.optim.lr_scheduler.StepLR
    _partial_: true
    step_size: 3
    gamma: 0.1
  compile: false
data:
  _target_: starfish.data.StarfishDataModule
  data_from_bucket: false
  batch_size: 32
  train_val_test_split:
  - 0.8
  - 0.1
  - 0.1
  subset: 1
  num_workers: 1
trainer:
  _target_: pytorch_lightning.Trainer
  min_epochs: 1
  max_epochs: 10
  accelerator: cpu
  devices: 1
  log_every_n_steps: 1
logger:
  _target_: pytorch_lightning.loggers.WandbLogger
  offline: false
  id: null
  anonymous: null
  project: Starfish Detection
  log_model: true
  prefix: ''
  entity: luciagordon-harvard-university
  group: ''
  tags:
  - test
  job_type: ''
profiling: false



NameError: name 'data' is not defined

In [2]:
data = instantiate(cfg_train.data)

data.setup()
train_dataloader = data.train_dataloader()


Loading 4919 images.


In [6]:
start = time.time()


for epoch in range(num_epochs):
    
    i=0

    for batch in train_dataloader:
        i+=1
        images, targets = batch
        # move images and targets to the device
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # output = model(images,targets)  # Forward pass

        if i%10==0:
            break


print(f"Time taken: {time.time()-start}")

Time taken: 132.4173140525818


In [4]:
from torchvision import models, datasets, ops
model = models.detection.ssdlite320_mobilenet_v3_large(weights="DEFAULT")
device = torch.device("mps") # torch.device("mps") if torch.backends.mps.is_available() else 
model.to(device)

model.train() 

num_epochs = 1  # Number of epochs to train for


# time it
import time
start = time.time()


for epoch in range(num_epochs):
    
    i=0

    for batch in train_dataloader:
        i+=1
        images, targets = batch
        # move images and targets to the device
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        output = model(images,targets)  # Forward pass

        if i%10==0:
            break


print(f"Time taken: {time.time()-start}")

Time taken: 99.11434197425842
